<a href="https://colab.research.google.com/github/adataschultz/MaritimeTrade_COVID/blob/main/MLP_Regression/MaritimeTradeCOVID_MLP_Train2019_2020_batch16_Epochs50_HPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
######################################################
#### 2019 - 2020 Maritime Trade with COVID - MLP #####
##########  Hyperparameter Optimization ##############
######################################################

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Data_MaritimeTrade_MLP/

/content/drive/MyDrive/Data_MaritimeTrade_MLP


In [4]:
# Install and import packages
!pip install category_encoders
!pip install keras-tuner
from tensorflow import keras
import keras_tuner
from keras_tuner import BayesianOptimization
import os
import random
import pandas as pd
import numpy as np
import category_encoders as ce
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import datetime
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib as plt
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print('\n')
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution is: {}".format(tf.executing_eagerly()))
print("Keras version: {}".format(tf.keras.__version__))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

     |████████████████████████████████| 80 kB 5.0 MB/s 
     |████████████████████████████████| 97 kB 5.0 MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm




TensorFlow version: 2.6.0
Eager execution is: True
Keras version: 2.6.0
Num GPUs Available:  1


In [5]:
# Reproducibility in TF and Pytorch 
def init_seeds(seed=1920):
    np.random.seed(seed)
    random.seed(seed)
    session_conf = tf.compat.v1.ConfigProto()
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
    os.environ['TF_CUDNN_DETERMINISTIC'] ='true'
    os.environ['TF_DETERMINISTIC_OPS'] = 'true'
    tf.random.set_seed(seed)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    return sess

In [6]:
init_seeds(seed=1920)


In [7]:
df = pd.read_csv('combined_trade_final_LSTM.csv', low_memory=False)
print('Number of rows and columns:', df.shape)

Number of rows and columns: (6633785, 20)


In [8]:
# Convert dtypes
df = df.copy()
df['State_Closure_EA_Diff'] = df['State_Closure_EA_Diff'].astype('float64')
df['Container_Type_Dry'] = df['Container_Type_Dry'].astype('object')

In [9]:
df = df.drop(['DateTime'], axis=1)

In [10]:
# Reformat for X,y
df2 = df.drop(['Metric_Tons'], axis=1)
df1 = df.loc[:, ['Metric_Tons']]
df = pd.concat([df1, df2], axis=1)

del df1, df2

In [11]:
# X,y for encoding
X = df.drop(['Metric_Tons'], axis=1)
y = df.iloc[:,:1]
print(X.shape)
print(y.shape)

(6633785, 18)
(6633785, 1)


In [12]:
# Set up train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify= X.Year, random_state = 1920)

In [13]:
# Training set: Encode variables using ranking - ordinal               
ce_ord = ce.OrdinalEncoder(cols = ['foreign_company_size', 'us_company_size'])
X_train = ce_ord.fit_transform(X_train, y_train['Metric_Tons'])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [14]:
# Testing set: Encode variables using ranking - ordinal               
ce_ord = ce.OrdinalEncoder(cols = ['foreign_company_size', 'us_company_size'])
X_test = ce_ord.fit_transform(X_test, y_test['Metric_Tons'])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [15]:
# Train: Create dummy variables for categorical variables
X_train = pd.get_dummies(X_train, drop_first=True)
X_train.head()

,TCVUSD,Year,us_company_size,foreign_company_size,US_Unemployment_Rate,Average_Tariff,cases_weekly,deaths_weekly,State_Closure_EA_Diff,cases_pctdelta,deaths_pctdelta,Delta_Case0_Effective,Container_LCL/FCL_LCL,Container_Type_Dry_True,Trade_Direction_Import,US_Port_Coastal_Region_Northeast,US_Port_Coastal_Region_Northwest,US_Port_Coastal_Region_Other,US_Port_Coastal_Region_Southeast,US_Port_Coastal_Region_Southwest,Foreign_Country_Region_European Union,Foreign_Country_Region_Middle East & North Africa,Foreign_Country_Region_North America,Foreign_Country_Region_Oceania,Foreign_Country_Region_Other East Asia (not China),Foreign_Country_Region_Other Europe (not European Union),Foreign_Country_Region_South America,Foreign_Country_Region_South Asia,Foreign_Country_Region_Southeast Asia,Foreign_Country_Region_Sub Saharan Africa,HS_Group_Name_Edible with Processing,HS_Group_Name_Finished Goods,HS_Group_Name_Pharma,HS_Group_Name_Raw Input,HS_Group_Name_Vices
1322785,14796.09,2020,1,1,14.8,13.24,242840.0,9105.0,2.0,24283900.0,910400.0,39.0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3187981,25621.34,2019,2,2,4.0,31.79,0.0,0.0,0.0,0.0,0.0,59.0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1062127,13326.58,2020,3,3,11.1,3.56,1447771.0,40865.0,0.0,144777000.0,4086400.0,59.0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1264197,113.67,2020,3,1,7.8,13.24,4867827.0,96828.0,2.0,486782600.0,9682700.0,39.0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5144839,63815.70,2019,4,3,3.6,11.03,0.0,0.0,2.0,0.0,0.0,27.0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [16]:
print('Dimensions of X_train for input:', X_train.shape[1])

Dimensions of X_train for input: 35


In [17]:
# Test: Create dummy variables for categorical variables
X_test = pd.get_dummies(X_test, drop_first=True)
X_test.head()

,TCVUSD,Year,us_company_size,foreign_company_size,US_Unemployment_Rate,Average_Tariff,cases_weekly,deaths_weekly,State_Closure_EA_Diff,cases_pctdelta,deaths_pctdelta,Delta_Case0_Effective,Container_LCL/FCL_LCL,Container_Type_Dry_True,Trade_Direction_Import,US_Port_Coastal_Region_Northeast,US_Port_Coastal_Region_Northwest,US_Port_Coastal_Region_Other,US_Port_Coastal_Region_Southeast,US_Port_Coastal_Region_Southwest,Foreign_Country_Region_European Union,Foreign_Country_Region_Middle East & North Africa,Foreign_Country_Region_North America,Foreign_Country_Region_Oceania,Foreign_Country_Region_Other East Asia (not China),Foreign_Country_Region_Other Europe (not European Union),Foreign_Country_Region_South America,Foreign_Country_Region_South Asia,Foreign_Country_Region_Southeast Asia,Foreign_Country_Region_Sub Saharan Africa,HS_Group_Name_Edible with Processing,HS_Group_Name_Finished Goods,HS_Group_Name_Pharma,HS_Group_Name_Raw Input,HS_Group_Name_Vices
5106325,60440.23,2019,1,1,3.6,2.28,0.0,0.0,2.0,0.0,0.0,27.0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6438450,48273.18,2020,2,2,14.8,18.80,11828.0,245.0,16.0,1182700.0,24400.0,30.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1831211,26647.72,2020,1,1,8.4,3.56,2969878.0,226578.0,2.0,296987700.0,22657700.0,27.0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3384697,23730.50,2019,1,1,3.7,18.80,0.0,0.0,0.0,0.0,0.0,59.0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1260139,0.00,2020,1,3,6.9,29.76,4980900.0,101462.0,2.0,498089900.0,10146100.0,39.0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [18]:
# MinMax Scaling
mn = MinMaxScaler()
X_train = pd.DataFrame(mn.fit_transform(X_train))
X_test = pd.DataFrame(mn.fit_transform(X_test))

In [19]:
log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

%load_ext tensorboard

In [20]:
# Set up callbacks
filepath = 'MLP_weights_only_train1920_b16_HPO1.h5'

checkpoint_dir = os.path.dirname(filepath)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_folder,
                                                      histogram_freq=1)

callbacks_list = [EarlyStopping(monitor='val_loss', patience = 5),
                  ModelCheckpoint(filepath, monitor='mse', 
                                  save_best_only = True, mode='min'), 
                  tensorboard_callback]

In [21]:
# Define model for HPO
def build_model(hp): 
  model = keras.Sequential()
  for i in range(hp.Int("num_layers", 2, 5)):
        model.add(tf.keras.layers.Dense( units=hp.Int("layer_size" + str(i), min_value=20, max_value=40, step=5),
                activation="relu", kernel_initializer='normal'))
  model.add(tf.keras.layers.Dropout(0.3))
  model.add(Dense(1))
  model.compile(loss="mae", metrics=["mse"], optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])))
  return model

In [22]:
# Define the search conditions 
tuner = BayesianOptimization(
    build_model,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=3,
    overwrite=True,
    directory="MLP_1920_HPO1",
    project_name="MLP_1920_HPO1"
)

In [23]:
# Print a summary of the search space
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': None}
layer_size0 (Int)
{'default': None, 'conditions': [], 'min_value': 20, 'max_value': 40, 'step': 5, 'sampling': None}
layer_size1 (Int)
{'default': None, 'conditions': [], 'min_value': 20, 'max_value': 40, 'step': 5, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [24]:
# Begin the search for the best hyperparameters 
tuner.search(X_train, y_train, epochs=3, validation_split=0.2, batch_size=16,
             callbacks=callbacks_list)

Trial 10 Complete [01h 25m 31s]
val_loss: 11.799287796020508

Best val_loss So Far: 11.799287796020508
Total elapsed time: 14h 01m 56s
INFO:tensorflow:Oracle triggered exit


In [25]:
# Retrieve the best model(s)
models = tuner.get_best_models(num_models=2)

In [26]:
# Print a summary of the results from the trials
tuner.results_summary()

Results summary
Results in MLP_1920_HPO1/MLP_1920_HPO1
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
num_layers: 4
layer_size0: 25
layer_size1: 25
learning_rate: 0.001
layer_size2: 25
layer_size3: 20
layer_size4: 20
Score: 11.799287796020508
Trial summary
Hyperparameters:
num_layers: 4
layer_size0: 25
layer_size1: 25
learning_rate: 0.001
layer_size2: 20
layer_size3: 20
Score: 12.087239583333334
Trial summary
Hyperparameters:
num_layers: 4
layer_size0: 20
layer_size1: 30
learning_rate: 0.001
layer_size2: 20
layer_size3: 20
layer_size4: 20
Score: 12.21741803487142
Trial summary
Hyperparameters:
num_layers: 5
layer_size0: 25
layer_size1: 30
learning_rate: 0.0001
layer_size2: 25
layer_size3: 40
layer_size4: 20
Score: 12.3302640914917
Trial summary
Hyperparameters:
num_layers: 2
layer_size0: 25
layer_size1: 35
learning_rate: 0.001
layer_size2: 20
layer_size3: 20
layer_size4: 20
Score: 12.37261931101481
Trial summary
Hyperparameters:
num_la